MNIST数据集的jax实现（此处用的是tensorflow）

In [2]:
!pip install jax tensorflow
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple tensor flow_datasets

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 17.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 14.1 MB/s eta 0:00:00a 0:00:01
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/5d/ae/3257b09328c0b4e59535e497b0c7537d4954038bdd53a2f0d2f49d15a7c4/protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/ba/a8/c17acf65a931ce551fee11b72e8de63bf7e8a6f0e21add4c937c83563538/numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━

In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds 
import jax.numpy as jnp
x_train = jnp.load("mnist_train_x.npy") 
y_train = jnp.load("mnist_train_y.npy")

In [ ]:
def one_hot_nojit(x, k=10, dtype=jnp. float32): 
         return jnp.array(x[:, None] == jnp.arange(k), dtype)

In [ ]:
# {Dense(1024) -> ReLU}x2 -> Dense(10) -> Logsoftmax 
init_random_params, predict = stax.serial(     
    stax.Dense(1024), stax.Relu,
    stax.Dense(1024), stax.Relu,
    stax.Dense(10), stax.Logsoftmax)

In [ ]:
import tensorflow as tf 
import tensorflow.datasets as tfds 
import jax 
import jax.numpy as jnp     
from jax import jit, grad, random     
from jax.experimental import optimizers     
from jax.experimental import stax     
num_classes = 10     
reshape_args = [(-1, 28 * 28), (-1,)]     
input_shape = reshape_args[0]     
step_size = 0.001     
num_epochs = 10     
batch_size = 128     
momentum_mass = 0.9     
rng = random.PRNGKey(0)     
x_train = jnp.load('mnist_train_x.npy')     
y_train = jnp.load('mnist_train_y.npy')

def one_hot_nojit(x, k=10, dtype=jnp. float32):          
    return jnp.array(x[:, None] == jnp.arange(k), dtype)     
total_train_imgs = len(y_train)

y_train = one_hot_nojit(y_train)
ds_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1024).batch(256).prefetch(tf.data.experimental.AUTOTUNE)
ds_train = tfds.as_numpy(ds_train)
def pred_check(params, batch):
    inputs, targets = batch          
    predict_result = predict(params, inputs)          
    predicted_class = jnp.argmax(predict_result, axis=1)
    targets = jnp.argmax(targets, axis=1)
    return jnp.sum(predicted_class == targets)
init_random_params, predict = stax.serial(stax.Dense(1024), stax.Relu, stax.Dense(1024), stax.Relu, stax.Dense(10), stax.Logsoftmax)     
def loss(params, batch):          
              inputs, targets = batch          
              return jnp.mean(jnp.sum(-targets * predict(params, inputs), axis=1))     
def update(i, opt_state, batch):               
    params = get_params(opt_state)
    return opt_update(i, grad(loss)(params, batch), opt_state)        
opt_init, opt_update, get_params = optimizers.adam(step_size = 2e-4) , init_params = init_random_params(rng, input_shape)     
opt_state = opt_init(init_params)
for _ in range(17):
         itercount = 0     
         for batch_raw in ds_train:          
               data = batch_raw[0].reshape((-1, 28 * 28))          
               targets = batch_raw[1].reshape((-1, 10)) 
               opt_state = update((itercount), opt_state, (data, targets)) 
               itercount += 1 
               params = get_params(opt_state)     
               train_acc = []     
               correct_preds = 0.0     
               for batch_raw in ds_train:
                          data = batch_raw[0].reshape((-1, 28 * 28))
                          targets = batch_raw[1]
                          correct_preds += pred_check(params, (data,) target)
               train_acc.append(correct_preds / float(total_train_imgs))     
               print(f'Training set accuracy: {train_acc}')
